# Perceptron - Basic Neural Networks with dense layers

We'll use SciKit Learn's built in Breast Cancer Data Set which has several features of tumors with a labeled class indicating whether the tumor was Malignant or Benign. We will try to create a neural network model that can take in these features and attempt to predict malignant or benign labels for tumors it has not seen before.

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

The return type of the build-in scikit data is "Bunch", a dictionary that supports attribute-style access, a la JavaScript

In [2]:
type(cancer)

sklearn.utils.Bunch

In [3]:
print(f'Keys={cancer.keys()}')

Keys=dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


You can access the keys as either dictionaries or attributes

In [4]:
print(f'feature_names:{cancer.feature_names}')
#print(f'feature_names:{cancer["feature_names"]}')

feature_names:['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [5]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [6]:
data = cancer['data']
data.shape

(569, 30)

In [7]:
target = cancer['target']
target.shape

(569,)

It is good practice to test control the data sizes before diving into ML

In [8]:
assert data.shape[0] == target.shape[0]
assert len(cancer.feature_names) == data.shape[1]

## Exercise 1

1) What information is provided in target?

2) What does it mean if the target value is 0?

3) Are the majority of the samples malignent or benign?

In [21]:
#1: target is 0, 1
#2: 0 means malignant 
#3: majority benign, but not by a lot
cancer['target_names']


array(['malignant', 'benign'], dtype='<U9')

## Exercise 2
In many cases, it is adventageous to convert the dictionary into a pandas object for a quick analysis.

Exercise: Create a Dataframe with both features and target data and use describe() to obtain a simple statistical report of the data

In [69]:
import pandas as pd
dataframe = pd.DataFrame(np.c_[data, target], columns=np.append(cancer.feature_names,'target'))
#targetframe = pd.DataFrame(target)
dataframe.describe()
?pd.DataFrame

## Create a Perceptron Prediction model

In [34]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30))
mlp.fit(data,target)

MLPClassifier(hidden_layer_sizes=(30, 30))

In [36]:
predictions = mlp.predict(data)

In [37]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(target,predictions))
print(classification_report(target,predictions))

[[187  25]
 [ 15 342]]
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       212
           1       0.93      0.96      0.94       357

    accuracy                           0.93       569
   macro avg       0.93      0.92      0.92       569
weighted avg       0.93      0.93      0.93       569



## Exercise 1:
But hey, we cheated 

1. Properly evaluate the model by splitting it into training and testing data.
2. Does data normalization improve the prediction?
3. Does it help to insert more layers?
4. What activation function is used by default? What happens if you change it?

1) Split into training and test data

In [176]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target)
assert X_train.shape[0] == len(y_train)
assert X_test.shape[0] == len(y_test)


2a) Non-normalized prediction

In [189]:
mlp = MLPClassifier(hidden_layer_sizes=(30,30))
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[48 12]
 [ 5 78]]
              precision    recall  f1-score   support

           0       0.91      0.80      0.85        60
           1       0.87      0.94      0.90        83

    accuracy                           0.88       143
   macro avg       0.89      0.87      0.88       143
weighted avg       0.88      0.88      0.88       143



2b) Normalized prediction

In [190]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
mlp = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(hidden_layer_sizes=(30,30), max_iter=1000))])
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[56  4]
 [ 1 82]]
              precision    recall  f1-score   support

           0       0.98      0.93      0.96        60
           1       0.95      0.99      0.97        83

    accuracy                           0.97       143
   macro avg       0.97      0.96      0.96       143
weighted avg       0.97      0.97      0.96       143



3) More layers

In [200]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
mlp = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(hidden_layer_sizes=(60,30,20,20), max_iter=1000))])
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[57  3]
 [ 1 82]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.97        60
           1       0.96      0.99      0.98        83

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143



In [120]:
?MLPClassifier

In [213]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate


mlp = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(hidden_layer_sizes=(30,30), activation='identity', max_iter=1000))])
mlp.fit(X_train, y_train)

print(scores)
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

{'fit_time': array([0.21649957, 0.20225835, 0.16907454, 0.23235106, 0.16663742]), 'score_time': array([0.0003581 , 0.00038052, 0.00035095, 0.00040936, 0.00036192]), 'test_score': array([0.98837209, 0.96470588, 0.98823529, 0.96470588, 0.98823529])}
[[55  5]
 [ 2 81]]
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        60
           1       0.94      0.98      0.96        83

    accuracy                           0.95       143
   macro avg       0.95      0.95      0.95       143
weighted avg       0.95      0.95      0.95       143

